<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/squad_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# looking at the squad dataset
# getting some of the imports
import requests
import json
import pandas as pd


In [2]:
files = ['train-v2.0.json', 'dev-v2.0.json']

In [3]:
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/"

In [4]:
# now getting the files 
for file in files:
  res = requests.get(url + file)
  if res.status_code == 200:
    
    with open(file, mode="wb") as f:
     # writing as a chunks 
     for chunk in res.iter_content(chunk_size=50):
       f.write(chunk)



In [5]:
# reading in one of the files to be used with json
with open("dev-v2.0.json", mode="rb")as f:
  dev = json.load(f)

with open("train-v2.0.json", mode="rb")as f:
  train = json.load(f)

In [6]:
train.keys()

dict_keys(['version', 'data'])

In [ ]:
train["data"][0]["title"]

'Beyoncé'

In [ ]:
type(train["data"][0]["paragraphs"])

list

In [7]:
squad_list = []

Getting the data into a format that can be used

In [8]:
# now getting the info out of the train
for paragraph_dict in train["data"]:
  for context_with_qa_dict in paragraph_dict["paragraphs"]:
    context = context_with_qa_dict["context"]
    # doing the looping through the question and the answers dictionaries
    for qa_pair_dict in context_with_qa_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
          answer = qa_pair_dict['answers'][0]['text']
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer = qa_pair_dict["plausible_answers"][0]["text"]
      else:
        answer = None
      # now making the dictionary that will be added to the list 
      squad_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})
        
      


In [9]:
len(squad_list)

130319

In [10]:
# creating the dataframe
df = pd.DataFrame(squad_list)

In [11]:
print(df.shape)
df.head()

(130319, 3)


,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s


In [12]:
df.loc[0, "context"]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [13]:
# now will save the data into a json type object
with open("train.json", mode="w", )as f:
  json.dump(squad_list, f)

In [14]:
dev_list = []

In [15]:
# doing the dev to make it so that it is in the same format as the 
# train.
for paragraph_dict in dev["data"]:
  for each_context_dict in paragraph_dict["paragraphs"]:
    context = each_context_dict["context"]
    for qa_pair_dict in each_context_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
        # doing the looping through the answers
        answer_list = qa_pair_dict["answers"]
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer_list = qa_pair_dict["plausible_answers"]
      else:
        answer_list = []
      # now doing the making of the list  
      answer = [item["text"] for item in answer_list]
      # removing any duplicates
      answer = list(set(answer))
      
      # adding to the dev list
      dev_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})


In [16]:
len(dev_list)

11873

In [17]:
# building a dataframe of the dev_list
dev_dataframe = pd.DataFrame(dev_list)
dev_dataframe.head()

,context,question,answer
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,[France]
1,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"[in the 10th and 11th centuries, 10th and 11th..."
2,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"[Denmark, Iceland and Norway]"
3,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,[Rollo]
4,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"[the first half of the 10th century, 10th cent..."


In [18]:
# making a json file of the new formatted dev
with open("dev.json", mode="w")as f:
  json.dump(dev_list, f)

In [28]:
# now making the first model 
! pip install transformers -q
import transformers
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import pipeline

In [26]:
# making the model
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

In [ ]:
# going to do it two ways -- one with using the pipelines and the other without the pipeline in 
# transformers
